### Dataset link: https://drive.grand-challenge.org/

Download data from https://drive.google.com/file/d/17wVfELqgwbp4Q02GD247jJyjq6lwB0l6/view

In [ ]:
# Import libraries

import os
import matplotlib.pyplot as plt
%matplotlib inline

from skimage.transform import resize
from skimage import io
import tifffile as tiff
import numpy as np
from mpl_toolkits.axes_grid1 import make_axes_locatable

ROOT_DIR = os.path.abspath("../")
DATASET_NAME = "DRIVE"
DATASET_PATH = os.path.join(ROOT_DIR, "datasets", DATASET_NAME)
print(os.listdir(DATASET_PATH))


#Data tree
#│
#└───test
#|    ├───1st_manual
#|    └───2nd_manual
#|    └───images
#|    └───mask
#│
#└───training
#    ├───1st_manual
#    └───images
#    └───mask
#└───all         # Combine train and test manually
#   └───1st_manual # 40 vessel masks
#   └───images # 40 images

In [ ]:
# Read train data
train_img_files = next(os.walk("{}/all/images/".format(DATASET_PATH)))[2]
train_msk_files = next(os.walk("{}/all/1st_manual/".format(DATASET_PATH)))[2]

train_img_files.sort()
train_msk_files.sort()

print(len(train_img_files))
print(len(train_msk_files))

In [ ]:
IMG_DIM = (512, 512)

images_rs = []
masks_rs = []

for img_fl, mask_fl in zip(train_img_files, train_msk_files):    
    
    img = tiff.imread('{}/all/images/{}'.format(DATASET_PATH, img_fl))
    img = resize(img, IMG_DIM, mode='constant')
    images_rs.append(img)


    mask = io.imread('{}/all/1st_manual/{}'.format(DATASET_PATH, mask_fl))
    mask = np.expand_dims(resize(mask, IMG_DIM).round(), axis=-1)
    masks_rs.append(mask)

    
images_rs = np.array(images_rs)
masks_rs = np.array(masks_rs)
images_rs.shape, masks_rs.shape

In [ ]:
for img, mask in zip(images_rs[:10], masks_rs[:10]):
    
    mask = np.squeeze(mask, axis=-1) # change to H, W
    
    
    fig,_ = plt.subplots(nrows=1, ncols=2, figsize=(14,12))
    fig.axes[0].imshow(img)
    fig.axes[1].imshow(mask, cmap='gray')
    plt.show()

In [ ]:
plt.rcParams['figure.figsize'] = (20, 5)
plt.title("Example images")
plt.figsize=(14,12)
plt.imshow(np.concatenate(images_rs[:5].transpose((0,1,2,3)),axis=1), interpolation='none')

In [ ]:
fig = plt.figure(1, figsize=(20, 5))
ax = plt.gca()
im = ax.imshow(np.concatenate(masks_rs[:5,:,:,0],axis=1), cmap='magma', interpolation='none') # cmap='bwr'
divider = make_axes_locatable(ax) 
cax = divider.append_axes("right", size="5%", pad=0.1)
plt.colorbar(im, cax=cax)

In [ ]:
from skimage.segmentation import mark_boundaries, find_boundaries

In [ ]:
ct = 5
edges_pz = mark_boundaries(images_rs[ct], masks_rs[ct].squeeze().astype(np.int),color=(1,0,0), mode='thin')
plt.imshow(edges_pz)

In [ ]:
images_rs.shape, masks_rs.shape

In [ ]:
np.save("{}/DRIVE_images.npy".format(DATASET_PATH), images_rs)
np.save("{}/DRIVE_mask.npy".format(DATASET_PATH), masks_rs)